##Movies Collaborative filtering using Spark

This example shows how to make collaborative filtering using Spark engine. In this notebook we use open MovieLens dataset (https://grouplens.org/datasets/movielens/).
First we need to download the dataset, unzip it and bring into the cluster storage.

In [2]:
%sh

export DATASET_NAME=ml-1m

rm -rf /tmp/movie-lens
wget -O /tmp/movie-lens.zip http://files.grouplens.org/datasets/movielens/${DATASET_NAME}.zip
unzip /tmp/movie-lens.zip -d /tmp/
rm -f /tmp/movie-lens.zip
mv /tmp/${DATASET_NAME} /tmp/movie-lens

--2020-03-05 14:16:48-- http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘/tmp/movie-lens.zip’

 0K .......... .......... .......... .......... .......... 0% 626K 9s
 50K .......... .......... .......... .......... .......... 1% 1.22M 7s
 100K .......... .......... .......... .......... .......... 2% 89.3M 5s
 150K .......... .......... .......... .......... .......... 3% 201M 3s
 200K .......... .......... .......... .......... .......... 4% 1.23M 4s
 250K .......... .......... .......... .......... .......... 5% 97.3M 3s
 300K .......... .......... .......... .......... .......... 6% 153M 3s
 350K .......... .......... .......... .......... .......... 6% 124M 2s
 400K .......... .......... .......... .......... .......... 7% 1.25M 2s
 450K .......... .......... .......... .......... .......... 8% 159M 2s
 500K .......... .......... .......... .......... .......... 9% 104M 2s
 550K .......... .......... .......... .......... .......... 10% 122M 2s
 600K .......... .......... .......... .......... .......... 11% 39.4M 2s
 650K .......... .......... .......... .......... .......... 12% 81.7M 1s
 700K .......... .......... .......... .......... .......... 12% 167M 1s
 750K .......... .......... .......... .......... .......... 13% 169M 1s
 800K .......... .......... .......... .......... .......... 14% 138M 1s
 850K .......... .......... .......... .......... .......... 15% 1.35M 1s
 900K .......... .......... .......... .......... .......... 16% 116M 1s
 950K .......... .......... .......... .......... .......... 17% 155M 1s
 1000K .......... .......... .......... .......... .......... 18% 112M 1s
 1050K .......... .......... .......... .......... .......... 19% 194M 1s
 1100K .......... .......... .......... .......... .......... 19% 95.9M 1s
 1150K .......... .......... .......... .......... .......... 20% 96.1M 1s
 1200K .......... .......... .......... .......... .......... 21% 106M 1s
 1250K .......... .......... .......... .......... .......... 22% 147M 1s
 1300K .......... .......... .......... .......... .......... 23% 106M 1s
 1350K .......... .......... .......... .......... .......... 24% 93.0M 1s
 1400K .......... .......... .......... .......... .......... 25% 88.5M 1s
 1450K .......... .......... .......... .......... .......... 25% 76.6M 1s
 1500K .......... .......... .......... .......... .......... 26% 69.3M 1s
 1550K .......... .......... .......... .......... .......... 27% 163M 1s
 1600K .......... .......... .......... .......... .......... 28% 170M 1s
 1650K .......... .......... .......... .......... .......... 29% 138M 1s
 1700K .......... .......... .......... .......... .......... 30% 1.47M 1s
 1750K .......... .......... .......... .......... .......... 31% 109M 1s
 1800K .......... .......... .......... .......... .......... 32% 135M 1s
 1850K .......... .......... .......... .......... .......... 32% 131M 1s
 1900K .......... .......... .......... .......... .......... 33% 167M 1s
 1950K .......... .......... .......... .......... .......... 34% 149M 1s
 2000K .......... .......... .......... .......... .......... 35% 110M 1s
 2050K .......... .......... .......... .......... .......... 36% 129M 0s
 2100K .......... .......... .......... .......... .......... 37% 135M 0s
 2150K .......... .......... .......... .......... .......... 38% 113M 0s
 2200K .......... .......... .......... .......... .......... 38% 132M 0s
 2250K .......... .......... .......... .......... .......... 39% 107M 0s
 2300K .......... .......... .......... .......... .......... 40% 130M 0s
 2350K .......... .......... .......... .......... .......... 41% 93.6M 0s
 2400K .......... .......... .......... .......... .......... 42% 142M 0s
 2450K .......... .......... ..

In [3]:
%sh
# Fix the delimiter from :: to ,
sed -i 's/::/,/g' /tmp/movie-lens/ratings.dat
sed -i 's/::/,/g' /tmp/movie-lens/movies.dat

Now the data lives on the driver node. We need to move it into the cluster storage. We execute the node on Databricks platform and hence move our files into the `dbfs`.

In [5]:
%fs rm -r dbfs:/FileStore/movie-lens

res14: Boolean = true

In [6]:
%fs mv file:/tmp/movie-lens/ratings.dat dbfs:/FileStore/movie-lens/ratings.csv

res15: Boolean = true

In [7]:
%fs mv file:/tmp/movie-lens/movies.dat dbfs:/FileStore/movie-lens/movies.csv

res16: Boolean = true

In [8]:
%sh
rm -rf /tmp/movie-lens*

In [9]:
%fs ls dbfs:/FileStore/movie-lens

path,name,size
dbfs:/FileStore/movie-lens/movies.csv,movies.csv,163542
dbfs:/FileStore/movie-lens/ratings.csv,ratings.csv,21593504


###Data preparation
Let's now read the source csv files and create dataframes out of them

In [11]:
csv_files = {"ratings": "dbfs:/FileStore/movie-lens/ratings.csv", "movies": "dbfs:/FileStore/movie-lens/movies.csv"}

sqlContext.clearCache()
ratings = spark.read.csv(csv_files["ratings"], header="false", inferSchema="true").toDF("userId", "movieId", "rating", "timestamp")
movies = spark.read.csv(csv_files["movies"], header="false", inferSchema="true").toDF("movieId", "title", "genres")

ratings.cache()
movies.cache()

print("Ratings:")
ratings.show(n=3)
print("Movies:")
movies.show(n=3)

Ratings:
+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1193| 5|978300760|
 1| 661| 3|978302109|
 1| 914| 3|978301968|
+------+-------+------+---------+
only showing top 3 rows

Movies:
+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Animation|Childre...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows

Let's identify all user ids. There are thousands of user ids. Each Id is a number, so the total size of the array is not too big and it's possible to bring it to the driver node.
We create a "userId" to "index" mapping dictionary that we can later use to simplify vector creation. Each user becomes a unique index. Index is a value from 0 to the (number of users - 1). This allows us later to create a feature vector for each movie, where a user rate is set to it's "index" position.

In [13]:
user_ids = ratings.select(ratings.userId).distinct().collect()

num_users = len(user_ids)
user_id_index_mapping = {}

for index in range(num_users):
  user_id_index_mapping[user_ids[index][0]] = index 

print("Number unique users %d" % num_users)

mapping_iter = iter(user_id_index_mapping.items())
print("First 3 mappings: {} {} {}".format(str(next(mapping_iter)), str(next(mapping_iter)), str(next(mapping_iter))))

Number unique users 6040
First 3 mappings: (148, 0) (463, 1) (471, 2)

Now we need to map all user rates to a corresponding movie. We store them into an array [(vector_index, user_rate), (vector_index, user_rate), ....]. Vector indexes are taken from a dictionary calculated on a previous step. In order to efficiently share the dictionary with all the worker nodes we create a broadcast variable out of it.

In [15]:
def make_movie_vector(movie_ratings):
  movie_id = movie_ratings[0]
  user_rates = []
  
  for user_id_rating in movie_ratings[1]:
    vector_index = user_id_index_mapping[user_id_rating[0]]
    user_rates.append((vector_index, user_id_rating[1]))
    
  return (movie_id, user_rates)

user_id_mapping_broadcast = sc.broadcast(user_id_index_mapping)
movie_rating_vectors = ratings.rdd.map(lambda row: (row.movieId, (row.userId, row.rating))).groupByKey().map(make_movie_vector).toDF(["movieId", "userRates"])

Let's also bring movie titles and genres into a result table.

In [17]:
movie_ratings = movie_rating_vectors.join(movies, movies.movieId == movie_rating_vectors.movieId).select(movie_rating_vectors.movieId, movie_rating_vectors.userRates, movies.title, movies.genres)
movie_ratings.cache()
movie_ratings.show(5)

+-------+--------------------+--------------------+--------------------+
movieId| userRates| title| genres|
+-------+--------------------+--------------------+--------------------+
 3408|[[1336, 4], [2004...|Erin Brockovich (...| Drama|
 594|[[1336, 4], [3734...|Snow White and th...|Animation|Childre...|
 720|[[1336, 3], [2688...|Wallace & Gromit:...| Animation|
 2340|[[1336, 3], [589,...|Meet Joe Black (1...| Romance|
 48|[[1336, 5], [1527...| Pocahontas (1995)|Animation|Childre...|
+-------+--------------------+--------------------+--------------------+
only showing top 5 rows

###Inference
Now we are done with all the data preparations. Let's create a similarity function.

Let's create a `find_similarities` function that gets a source movie id and returns a sorted dataframe with similarity scores. The function calculates similarities based on user rates between the source movie and all the other movies. The calculation goes though the following steps:

- Get a `source_vector` data frame that contains one row with user rates of a source movie.
- Make a cross join between the original `move_ratings` data frame and `source_vector`. This creates a new data frame that contains both source and movie vectors in each row.
- Run `calculate_similarity` for each row:
  - Build a sparse numpy vector of size "number of users" for source rates.
  - Build a sparse numpy vector of size "number of users" for movie rates.
  - Calculate cosine similarity between two vectors.
- Store calculated similarity scores into a separate column.
- Sort data frame in descending order.

In [19]:
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
import numpy as np
from numpy import dot
from numpy.linalg import norm


vector_size = num_users

def build_numpy_vector(rate_mappings):
  feature_vector = np.zeros(vector_size, dtype=float)
  
  for mapping in rate_mappings:
    feature_vector[mapping[0]] = mapping[1]
    
  return feature_vector

@udf("float")
def calculate_similarity(source_rates, movie_rates):
  source_vector = build_numpy_vector(source_rates)
  target_vector = build_numpy_vector(movie_rates)

  cos_sim = dot(source_vector, target_vector)/(norm(source_vector) * norm(target_vector))
  return cos_sim.item()

spark.udf.register("calculate_similarity", calculate_similarity)


def find_similarities(source_movie_id):
  source_vector = movie_ratings.select(col("userRates").alias("sourceRates")).filter(movie_ratings.movieId == source_movie_id)
  scores = movie_ratings.crossJoin(source_vector).select(movie_ratings.title, movie_ratings.genres, calculate_similarity("sourceRates", "userRates").alias("score"))
  return scores.orderBy(col("score"), ascending=False)

We are done with implementation. Now let's pick a movie and display the most similar ones.

In [21]:
source_movie_id = movie_ratings.filter(movie_ratings.title.contains("Star Wars: Episode I - The Phantom Menace")).select(movie_ratings.movieId).collect()[0].movieId

similarities = find_similarities(source_movie_id)
similarities.show(20, truncate=False)

+-----------------------------------------------------+-----------------------------------+----------+
title |genres |score |
+-----------------------------------------------------+-----------------------------------+----------+
Star Wars: Episode I - The Phantom Menace (1999) |Action|Adventure|Fantasy|Sci-Fi |1.0 |
Star Wars: Episode IV - A New Hope (1977) |Action|Adventure|Fantasy|Sci-Fi |0.6432094 |
Men in Black (1997) |Action|Adventure|Comedy|Sci-Fi |0.6338659 |
Matrix | The (1999) |0.6325046 |
Star Wars: Episode V - The Empire Strikes Back (1980)|Action|Adventure|Drama|Sci-Fi|War |0.62954915|
Star Wars: Episode VI - Return of the Jedi (1983) |Action|Adventure|Romance|Sci-Fi|War|0.6284211 |
Jurassic Park (1993) |Action|Adventure|Sci-Fi |0.6245523 |
Independence Day (ID4) (1996) |Action|Sci-Fi|War |0.6154788 |
Terminator 2: Judgment Day (1991) |Action|Sci-Fi|Thriller |0.6060164 |
Total Recall (1990) |Action|Adventure|Sci-Fi|Thriller |0.6037184 |
Star Trek: First Contact (1996) |Action|Adventure|Sci-Fi |0.590923 |
Fifth Element | The (1997) |0.57615143|
Stargate (1994) |Action|Adventure|Sci-Fi |0.57508034|
Starship Troopers (1997) |Action|Adventure|Sci-Fi|War |0.55159473|
Star Trek: Generations (1994) |Action|Adventure|Sci-Fi |0.5490058 |
Raiders of the Lost Ark (1981) |Action|Adventure |0.5470809 |
Terminator | The (1984) |0.54705924|
Back to the Future (1985) |Comedy|Sci-Fi |0.54569346|
Mission: Impossible (1996) |Action|Adventure|Mystery |0.5448104 |
Star Trek VI: The Undiscovered Country (1991) |Action|Adventure|Sci-Fi |0.5392705 |
+-----------------------------------------------------+-----------------------------------+----------+
only showing top 20 rows

Above is the list of the most similar movies. Obviously, the first one is the same movie with score `1.0`. It shows us that similarity calculation works as expected. 
To sum up, the above example shows how to build a simple collaborative filtering algorithm using Spark. The system allows to find movies similar to a source one. The recommendation here is done based on the rates given by users. So, the most similar films have the most number of similar scores given by same users.